In [1]:
import json
import requests
import pandas as pd

In [2]:
url = "https://api.openf1.org/v1/sessions?session_type=Race&year=2023"
response = requests.get(url)
print(f"Status Code: {response.status_code}")

Status Code: 200


In [3]:
data = response.json()
print(json.dumps(data[:3], indent=4))

[
    {
        "location": "Sakhir",
        "country_key": 36,
        "country_code": "BRN",
        "country_name": "Bahrain",
        "circuit_key": 63,
        "circuit_short_name": "Sakhir",
        "session_type": "Race",
        "session_name": "Race",
        "date_start": "2023-03-05T15:00:00+00:00",
        "date_end": "2023-03-05T17:00:00+00:00",
        "gmt_offset": "03:00:00",
        "session_key": 7953,
        "meeting_key": 1141,
        "year": 2023
    },
    {
        "location": "Jeddah",
        "country_key": 153,
        "country_code": "KSA",
        "country_name": "Saudi Arabia",
        "circuit_key": 149,
        "circuit_short_name": "Jeddah",
        "session_type": "Race",
        "session_name": "Race",
        "date_start": "2023-03-19T17:00:00+00:00",
        "date_end": "2023-03-19T19:00:00+00:00",
        "gmt_offset": "03:00:00",
        "session_key": 7779,
        "meeting_key": 1142,
        "year": 2023
    },
    {
        "location": "Melb

In [4]:
df = pd.DataFrame(data)
df.head(5)

,location,country_key,country_code,country_name,circuit_key,circuit_short_name,session_type,session_name,date_start,date_end,gmt_offset,session_key,meeting_key,year
0,Sakhir,36,BRN,Bahrain,63,Sakhir,Race,Race,2023-03-05T15:00:00+00:00,2023-03-05T17:00:00+00:00,03:00:00,7953,1141,2023
1,Jeddah,153,KSA,Saudi Arabia,149,Jeddah,Race,Race,2023-03-19T17:00:00+00:00,2023-03-19T19:00:00+00:00,03:00:00,7779,1142,2023
2,Melbourne,5,AUS,Australia,10,Melbourne,Race,Race,2023-04-02T05:00:00+00:00,2023-04-02T07:00:00+00:00,10:00:00,7787,1143,2023
3,Baku,30,AZE,Azerbaijan,144,Baku,Race,Sprint,2023-04-29T13:30:00+00:00,2023-04-29T14:00:00+00:00,04:00:00,9069,1207,2023
4,Baku,30,AZE,Azerbaijan,144,Baku,Race,Race,2023-04-30T11:00:00+00:00,2023-04-30T13:00:00+00:00,04:00:00,9070,1207,2023


In [5]:
# Fetching Code for Multiple Session Keys

session_keys = [7775, 7779, 9476, 9480, 9689, 9693, 9994, 9998]

endpoints = {
    "drivers": "https://api.openf1.org/v1/drivers",
    "position": "https://api.openf1.org/v1/position",
    "laps": "https://api.openf1.org/v1/laps",
    "sessions": "https://api.openf1.org/v1/sessions",
}

dataframes = {key: [] for key in endpoints.keys()}

for session_key in session_keys:
    for key, url in  endpoints.items():
        response = requests.get(url, params = {"session_key": session_key }, stream = True)
        if response.status_code == 200:
            df = pd.DataFrame(response.json())
            if not df.empty:
                dataframes[key].append(df)
        else:
            print(f"Failed to fetch {key} data, Status Code: ", response.status_code)

for key in dataframes:
    if dataframes[key]: dataframes[key] = pd.concat(dataframes[key], ignore_index = True)
    else: dataframes[key] = pd.DataFrame()

for key, df in dataframes.items():
    print(f"{key}: {df.shape}")
    

drivers: (160, 12)
position: (6099, 5)
laps: (5031, 16)
sessions: (8, 14)


In [6]:
# Extract Columns of Interest and Optimize Data

driver_subset = dataframes["drivers"][["session_key", "driver_number", "full_name", "team_name", "headshot_url"]]
position_subset = dataframes["position"][["session_key", "driver_number", "position"]]
laps_subset = dataframes["laps"][["session_key", "driver_number", "lap_number"]]
sessions_subset = dataframes["sessions"][["session_key", "circuit_key", "session_type", "location", "date_start"]]


# Optimize dataframes so we only get final positions of each session
position_subset = position_subset.sort_values(["session_key", "driver_number"]).drop_duplicates(subset = ["session_key", "driver_number"], keep = "last")

max_laps = laps_subset.groupby(["session_key", "driver_number"])["lap_number"].max().reset_index()
laps_subset = laps_subset.merge(max_laps, on=["session_key", "driver_number", "lap_number"], how="inner")


# Final array of dataframes before merging
dataframes_subset = [driver_subset, position_subset, laps_subset, sessions_subset]

print(f"Driver subset shape: {driver_subset.shape}")
print(f"Position subset shape: {position_subset.shape}")
print(f"Laps subset shape: {laps_subset.shape}")
print(f"Sessions subset shape: {sessions_subset.shape}")

def optimize_dataframe(df):
    
    # Create a copy of the dataframe
    df = df.copy()  

    # Convert int64 to int32
    for col in df.select_dtypes(include=["int64"]).columns:
        df.loc[:, col] = pd.to_numeric(df[col], downcast = "integer")  

    # Convert float64 to float32
    for col in df.select_dtypes(include=["float64"]).columns:
        df.loc[:, col] = pd.to_numeric(df[col], downcast = "float")

    # Convert object to category
    for col in df.select_dtypes(include=["object"]).columns:
        df.loc[:, col] = df[col].astype("category")

    return df

dataframes_subset = [optimize_dataframe(df) for df in dataframes_subset]

print(dataframes_subset[0].dtypes)

Driver subset shape: (160, 5)
Position subset shape: (160, 3)
Laps subset shape: (159, 3)
Sessions subset shape: (8, 5)
session_key       int64
driver_number     int64
full_name        object
team_name        object
headshot_url     object
dtype: object


In [7]:
# Merge DataFrames

# Start with the first DataFrame
merged_df = dataframes_subset[0]  
print(f"Initial dataframe shape: {merged_df.shape}")

for i in range(1, len(dataframes_subset)):  

    # Get rid of duplicate keys before each merge
    dataframes_subset[i] = dataframes_subset[i].drop_duplicates()

    # Determine common columns between the two DataFrames
    common_cols = list(set(merged_df.columns) & set(dataframes_subset[i].columns))
    
    # Ensure 'session_key' is always used as a key
    if "session_key" not in common_cols:
        common_cols.append("session_key")

    # Merge only on common columns
    merged_df = pd.merge(
        merged_df, dataframes_subset[i],
        on = common_cols,
        how = "inner"
    ).astype("category", errors = "ignore")

    merged_df = optimize_dataframe(merged_df)
    print(f"Merged Dataframe shape on {i}, {merged_df.shape}")

print(f"Final Merged DataFrame {merged_df.shape}:")
print(merged_df.columns)
merged_df["year"] = merged_df["date_start"].astype(str).str[:4]
merged_df = merged_df.drop(columns = ["date_start"])
print(merged_df.shape)
desired_order = ["session_key", "session_type", "position", "driver_number", "full_name", "team_name", "lap_number", "circuit_key", "location", "year", "headshot_url"]
final_df = merged_df[desired_order]
final_df

Initial dataframe shape: (160, 5)
Merged Dataframe shape on 1, (160, 6)
Merged Dataframe shape on 2, (159, 7)
Merged Dataframe shape on 3, (159, 11)
Final Merged DataFrame (159, 11):
Index(['session_key', 'driver_number', 'full_name', 'team_name',
       'headshot_url', 'position', 'lap_number', 'circuit_key', 'session_type',
       'location', 'date_start'],
      dtype='object')
(159, 11)


,session_key,session_type,position,driver_number,full_name,team_name,lap_number,circuit_key,location,year,headshot_url
0,7775,Qualifying,15,1,Max VERSTAPPEN,NaN,6,149,Jeddah,2023,NaN
1,7775,Qualifying,20,2,Logan SARGEANT,NaN,8,149,Jeddah,2023,NaN
2,7775,Qualifying,19,4,Lando NORRIS,NaN,4,149,Jeddah,2023,NaN
3,7775,Qualifying,10,10,Pierre GASLY,NaN,23,149,Jeddah,2023,NaN
4,7775,Qualifying,1,11,Sergio PEREZ,NaN,18,149,Jeddah,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...
154,9998,Race,6,44,Lewis HAMILTON,Ferrari,56,49,Shanghai,2025,https://media.formula1.com/d_driver_fallback_i...
155,9998,Race,13,55,Carlos SAINZ,Williams,56,49,Shanghai,2025,https://media.formula1.com/d_driver_fallback_i...
156,9998,Race,3,63,George RUSSELL,Mercedes,56,49,Shanghai,2025,https://media.formula1.com/d_driver_fallback_i...
157,9998,Race,1,81,Oscar PIASTRI,McLaren,56,49,Shanghai,2025,https://media.formula1.com/d_driver_fallback_i...


In [8]:
from pymongo import MongoClient

In [9]:
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_db"]
collection = db["race_data"]

In [10]:
data_dict = final_df.to_dict("records")

In [11]:
collection.insert_many(data_dict)

InsertManyResult([ObjectId('67ed8c8ed82bfcd9410eb148'), ObjectId('67ed8c8ed82bfcd9410eb149'), ObjectId('67ed8c8ed82bfcd9410eb14a'), ObjectId('67ed8c8ed82bfcd9410eb14b'), ObjectId('67ed8c8ed82bfcd9410eb14c'), ObjectId('67ed8c8ed82bfcd9410eb14d'), ObjectId('67ed8c8ed82bfcd9410eb14e'), ObjectId('67ed8c8ed82bfcd9410eb14f'), ObjectId('67ed8c8ed82bfcd9410eb150'), ObjectId('67ed8c8ed82bfcd9410eb151'), ObjectId('67ed8c8ed82bfcd9410eb152'), ObjectId('67ed8c8ed82bfcd9410eb153'), ObjectId('67ed8c8ed82bfcd9410eb154'), ObjectId('67ed8c8ed82bfcd9410eb155'), ObjectId('67ed8c8ed82bfcd9410eb156'), ObjectId('67ed8c8ed82bfcd9410eb157'), ObjectId('67ed8c8ed82bfcd9410eb158'), ObjectId('67ed8c8ed82bfcd9410eb159'), ObjectId('67ed8c8ed82bfcd9410eb15a'), ObjectId('67ed8c8ed82bfcd9410eb15b'), ObjectId('67ed8c8ed82bfcd9410eb15c'), ObjectId('67ed8c8ed82bfcd9410eb15d'), ObjectId('67ed8c8ed82bfcd9410eb15e'), ObjectId('67ed8c8ed82bfcd9410eb15f'), ObjectId('67ed8c8ed82bfcd9410eb160'), ObjectId('67ed8c8ed82bfcd9410eb1

In [12]:
for doc in collection.find().limit(5):
    print(doc)

{'_id': ObjectId('67db188a0cddb9887f2b6399'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 15, 'driver_number': 1, 'full_name': 'Max VERSTAPPEN', 'team_name': nan, 'lap_number': 6, 'circuit_key': 149, 'location': 'Jeddah', 'year': '2023', 'headshot_url': nan}
{'_id': ObjectId('67db188a0cddb9887f2b639a'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 20, 'driver_number': 2, 'full_name': 'Logan SARGEANT', 'team_name': nan, 'lap_number': 8, 'circuit_key': 149, 'location': 'Jeddah', 'year': '2023', 'headshot_url': nan}
{'_id': ObjectId('67db188a0cddb9887f2b639b'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 19, 'driver_number': 4, 'full_name': 'Lando NORRIS', 'team_name': nan, 'lap_number': 4, 'circuit_key': 149, 'location': 'Jeddah', 'year': '2023', 'headshot_url': nan}
{'_id': ObjectId('67db188a0cddb9887f2b639c'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 10, 'driver_number': 10, 'full_name': 'Pierre GASLY', 'team